In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
import csv
import random

### Trial Parsing

In [29]:
rDots_home_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/'
rDots_pcm_dir = os.path.join(rDots_home_dir, 'pcm/')
rDots_wav_dir = os.path.join(rDots_home_dir, 'wav/')
rDots_ndx_dir = os.path.join(rDots_home_dir, 'ndx/')
rDots_infos_dir = os.path.join(rDots_home_dir, 'infos/')

In [112]:
gender = 'm'
part_n = '1'
prefix = "{}_part{}".format(gender, str(part_n))

### ndx

In [113]:
def rDots_parse_ndx(ndx_dir, gender, part_num):
    assert (gender in ['f', 'm'])
    part_num = int(part_num[0])
    assert (part_num in [1,2,3,4])
    spk_sent_ID, file_names, trial_types = [], [], []
    
    with open(ndx_dir + gender + '_part_0' + str(part_num) + '.ndx', 'r') as ndx:
        lines = ndx.readlines()
        for line in lines:
            line = line.strip()
            [spk_sent, utterance, TC, TW, IC, IW] = line.split(',')
            file_names.append(utterance+".wav")
            spk_sent_ID.append(spk_sent)

            if TC=='Y': trial_types.append(0) 
            elif TW=='Y': trial_types.append(1) 
            elif IC=='Y': trial_types.append(2) 
            elif IW=='Y': trial_types.append(3) 
    return [spk_sent_ID, file_names,  trial_types]

In [114]:
rDots_ndx_dir = os.path.join(rDots_home_dir, 'ndx/')
spk_sent_ID, file_names,trial_types = rDots_parse_ndx(rDots_ndx_dir, gender, part_n)
ndx_dict = dict(id=spk_sent_ID, file=file_names, trial_type=trial_types)

In [115]:
ndx = pd.DataFrame.from_dict(ndx_dict)
manifest_dir = "manifests/reddots/trial/{}/".format(prefix)
if not os.path.isdir(manifest_dir):
    os.makedirs(manifest_dir)
ndx.to_pickle(os.path.join(manifest_dir, "{}_ndx.pkl".format(prefix)))

### trn

In [116]:
def rDots_parse_trn(trn_dir, gender, part_num):
    assert (gender in ['f', 'm'])
    assert (part_num in ['1','2','3','4_tp', '4_td'])
    spk_sent_ID, file_names = [], []

    with open(trn_dir + gender + '_part_0' + str(part_num) + '.trn', 'r') as trn:
        lines = trn.readlines()
        for line in lines:
            line = line.strip()
            [spk_sent, utters] = line.split(' ')
            utters = utters.split(',')
            for uttr in utters:
                spk_sent_ID.append(spk_sent)
                file_names.append(uttr+".wav")
    assert(len(spk_sent_ID) == len(file_names))
    return [spk_sent_ID, file_names]

In [117]:
rDots_trn_dir = os.path.join(rDots_home_dir, 'ndx/')
[spk_sent, file_names] = rDots_parse_trn(rDots_trn_dir, gender, part_n)
trn_dict = dict(id=spk_sent, file=file_names)

In [118]:
trn = pd.DataFrame.from_dict(trn_dict)
trn.to_pickle(os.path.join(manifest_dir,"{}_trn.pkl".format(prefix)))
trn_id = list(trn.id.unique())

## Adaptive Learning Scenario

In reddots,  
Trn and ndx files are disjoint. (In all parts)

TW samples are only 252 out of 3854 in "m0001_31" case.  

m_part4_tp is a good choice I think.


In [109]:
reddots_df = pd.read_pickle("dataset/dataframes/Reddots/Reddots_Dataframe.pkl")

In [139]:
trn = pd.read_pickle("dataset/dataframes/Reddots/m_part4_tp/m_part4_tp_trn.pkl")
ndx = pd.read_pickle("dataset/dataframes/Reddots/m_part4_tp/m_part4_tp_ndx.pkl")

In [144]:
ndx['spk'] = ndx.file.apply(lambda x: x.split("/")[0])

In [151]:
m0001_31_ndx = ndx[ndx.id=='m0001_31']

In [152]:
m0001_31_ndx.spk.value_counts()

m0001    624
m0002    598
m0009    432
m0020    408
m0013    407
m0007    312
m0040    288
m0015    288
m0008    240
m0026    240
m0022    216
m0029    216
m0048    192
m0053    192
m0014    192
m0005    168
m0017    168
m0021    168
m0006    168
m0043    168
m0028    120
m0047    120
m0050    120
m0054    120
m0032     96
m0010     96
m0067     96
m0018     96
m0060     96
m0038     72
m0041     72
m0016     72
m0052     72
m0064     48
m0051     48
m0023     48
m0037     48
m0057     48
m0030     24
m0025     24
m0055     24
m0004     24
m0035     24
m0062     24
m0059     24
m0056     24
m0063     24
m0019     24
m0045     24
Name: spk, dtype: int64

In [143]:
len(ndx[ndx.id == "m0001_31"])

7437

In [ ]:
n_enroll_uttrs = 2
positive_df = common_words_df[common_words_df.spk.isin(enroll_spks)] # utterances from enroll spks
grouped = positive_df.groupby(['spk', 'sent'], as_index=False, group_keys=False)
enroll_df = grouped.apply(lambda x: x.sample(n=n_enroll_uttrs)) # enroll utterances for each word